In [1]:
from datasets import load_dataset

from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM

import torch
from torch.utils.data import DataLoader
from torch.optim import AdamW

from model import Model

torch.set_printoptions(linewidth=1000000)

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/conda/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


# Load trained model
Load the model from a trained checkpoint

In [2]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [3]:
model = Model.load_from_checkpoint('flan-t5-base-batch4.ckpt')
tokenizer = AutoTokenizer.from_pretrained('google/flan-t5-base', truncation_side='left')
model.eval()

Model(
  (model): PeftModelForSeq2SeqLM(
    (base_model): LoraModel(
      (model): T5ForConditionalGeneration(
        (shared): Embedding(32128, 768)
        (encoder): T5Stack(
          (embed_tokens): Embedding(32128, 768)
          (block): ModuleList(
            (0): T5Block(
              (layer): ModuleList(
                (0): T5LayerSelfAttention(
                  (SelfAttention): T5Attention(
                    (q): Linear(
                      in_features=768, out_features=768, bias=False
                      (lora_dropout): ModuleDict(
                        (default): Dropout(p=0.05, inplace=False)
                      )
                      (lora_A): ModuleDict(
                        (default): Linear(in_features=768, out_features=16, bias=False)
                      )
                      (lora_B): ModuleDict(
                        (default): Linear(in_features=16, out_features=768, bias=False)
                      )
                      (lora_embeddi

# Configure dialog with model
Creating an action to easily "communicate" with the model

In [4]:
context = {}

def dialog(topic, prompt):
    with torch.no_grad():
        current_context = ''

        if topic in context:
            context[topic].append(prompt)
        else:
            context[topic] = []
            context[topic].append(prompt)

        current_context = ' '.join(context[topic])
        tokenized_input = tokenizer(current_context, return_tensors="pt").to(device)

        generated = model.model.generate(**tokenized_input, do_sample=True, top_p=0.9, max_new_tokens=200)
        print(generated.dtype)
        generated_text = tokenizer.decode(generated[0], skip_special_token=True)
        context[topic].append(generated_text)

        return generated_text
    

In [5]:
print(dialog('eval', 'What is the white house?'))

torch.int64
<pad> I am open to suggestions and discussions, but I am afraid that I am not fully prepared for the potential impact that the White House might have on our own or other peoples. There are several factors that could affect the White House: 1. The White House is a political institution and has been founded on the principles of the US Constitution. The White House is a federal institution of government, and is governed by the Constitution and other laws. 2. The Senate is the administrative body of government and is run by the Senate and House of Representatives. This includes a broad range of constitutional laws and regulations, including the United States Constitution, the American Constitution, and the United States Constitution. 3. The Supreme Court is a court of law and is governed by the House of Representatives and House of Representatives. 4. The Supreme Court is a court of law and is ruled by the Senate and House of Representatives. 5. The House of Representatives is 

In [6]:
print(dialog('eval', 'Who lives there?'))

torch.int64
<pad> It is governed by the American Constitution and other laws. The United States Constitution also grants access to certain regions of the United States that are part of the United States, including those located in countries that are members of the European Union or other third-country nations. Among the states that have a member state are: New Jersey: New Jersey, New Jersey: New Jersey, New Jersey: New Jersey, New Jersey: New Jersey, New Jersey: New Jersey, New Jersey: New Jersey, New Jersey, New Jersey: New Jersey, New Jersey: New Jersey, New Jersey: New Jersey, New Jersey: New Jersey, New Jersey, New Jersey: New Jersey, New Jersey: New Jersey, New Jersey: New Jersey, New Jersey: New Jersey, New Jersey: New Jersey, New Jersey: New Jersey, New Jersey: New Jersey, New Jersey: New Jersey, New Jersey: New Jersey, New Jersey: New Jersey, New Jersey: New Jersey, New Jersey:


In [7]:
print(dialog('python', 'What is python for?'))

torch.int64
<pad> The Python framework is used for writing and testing programs for Python. The framework has a modular approach, and the functions are built on top of each other, to allow the framework to be used in a wide range of applications. The framework allows users to define the Python programming framework using a graphical user interface (GUI), such as the Python script library and the Python interface, to make writing and testing program easier. The framework is also available as a plugin for Python, which allows users to create and use Python programs for their own programs, such as Python's GUI. The framework allows users to define the Python programming framework using the Python interface and the Python IDE, and has an application-specific interface for creating and deploying Python applications. This allows developers to easily create and deploy Python programs for use in Python's IDE, which are available on Python's website and on the Python programming framework. Ther

In [8]:
print(dialog('formel1', 'Who drives in F1?'))

torch.int64
<pad> I'm the driver in F1. It's my job to drive for the most advantage of my team, not for a paycheck. But I'm also responsible for managing our team, and supporting our drivers. I'm a team leader and I'm a team member. My job is to ensure I'm on the same level as other teams in F1. I can work with my team to ensure that they're in the same league and that they're able to get the support they need. I'm also responsible for managing my team's logistics and ensure that we keep track of important events and activities that need to be taken care of. I'm also responsible for managing my own team, and coordinating our own teamwork and resources. I'm responsible for managing my own team and supporting our own drivers, and I'm also responsible for managing the development and logistics of the team. I'm also responsible for keeping track


In [9]:
print(dialog('formel1', 'How many titles have you won?'))

torch.int64
<pad> I have won the following titles: 1. The <unk>Atana<unk>a F1<unk> circuits. This game has been compared with F1. 3. The <unk>Italian<unk>a F1<unk> circuits. This game has been compared to F1. 4. The <unk>Asiatic<unk>a F1<unk> circuits. This game has been compared to F1. 5. The <unk>Italian<unk>a F1<unk> circuits. This game has been compared to F1. 6. The <unk>F1, and its many different teams have been compared. This game has been compared to F1. 7. The <unk>F1, and its many different teams have been compared to F1. 8. The <unk>F1, and its many different teams have been compared to F1. 9. The <unk>F1, and its many different teams have been compared to F1. 10. The <unk>F1<unk> circuits. This game has been


In [10]:
print(dialog('formel1', 'how many races are in one season of formula 1?'))

torch.int64
<pad> As of February 4, if the drivers are in Formula 1 with the team members currently at the track, they are still able to continue their racing on the track as part of a season of formula 1. However, the formula 1 events will be updated at a later date for you to get updated in case of changes. 4. In Formula 1: The formula 1 event will be featured on the team's website, and will be displayed in the aforementioned article. The afno, afno, and efno are all featured in Formula 1 games in season 3. In Formula 1, Formula 1 will also feature an updated Formula 1 show on July 5, which can be updated every day until June 25. 5. Formula 2: Formula 3 will feature a series of Formula 1 events in season 5. Formula 4 will feature a series of Formula 1 events in season 6. Formula 5 will feature a series of Formula 1 events in season 7. Formula 5


In [11]:
print(dialog('word', 'How can I use microsoft word?'))

torch.int64
<pad> How to use Microsoft Word in a number of ways: 1. To make a new document from the beginning of the document, you can open the document and save it as a new document. 2. Open the document, select the new document and save the document as a new document. 3. Save the document and save the document as a new document. 4. Publish the new document. The new document will be created automatically if you save the new document. 5. Insert the new document into the document. 6. Convert the new document into a new document. 7. Save the new document and send it to your friends and family members. 8. Set your bookmarks and add the new document. 9. Edit the current document to your preferred format. 10. Close the new document and save the document. 11. Publish the new document with the new document. 12. Open the new document and delete the new document. 13. Close the new document and save the document. 14. Add the new document to


In [12]:
print(dialog('word', 'Do I have to buy it to use it?'))

torch.int64
<pad> Yes You can buy a copy of Microsoft Word or use it as a free edition. Here's an example of how you can buy Microsoft Word: 1. Download a copy of Microsoft Word: Download the app from the website of Microsoft Word: Click the "Download" button and follow the prompts to install the software. 2. Open the Microsoft Word document: Select the Microsoft Word document from the list of available languages and click "Controll". 3. Copy the Microsoft Word document. Note that the new document may require a few settings and other changes before being used. 4. Select a bookmark: Select the bookmark to save your new document. 5. Convert the new document into a new document. 6. Set your bookmarks and add the new document. 7. Set your bookmarks and add the new document. 8. Set your bookmarks and add the new document. 9. Edit the current document to your preferred format. 10. Close the new document and save the document. 11. Add the


In [13]:
print(dialog('word', 'Where can I buy it?'))

torch.int64
<pad> I am currently trained as a professional software designer. I am currently not a qualified professional but I can help you with your certifications and background. 1. Download a copy of Microsoft Word: You can download the app from the website of Microsoft Word: Click the "Download" button and follow the prompts to install the software. 2. Open the Microsoft Word document: Select the Microsoft Word document from the list of available languages and click "Controll". 3. Copy the Microsoft Word document. Note that the new document may require a few settings and other changes before being used. 4. Set your bookmarks and add the new document. 5. Convert the new document into a new document. 6. Set your bookmarks and add the new document. 7. Set your bookmarks and add the new document. 8. Set your bookmarks and add the new document. 9. Edit the current document to your preferred format. 10. Close the new document and save the document. 11. Add the new document to Add
